This notebook contains a notebook version of the finetune process. We'll do exactly the same but using GCP instances.

In [1]:
!pip install datasets unsloth xformers

In [2]:
from huggingface_hub import notebook_login

notebook_login()

First of all, we are going to load the dataset containing Rick & Morty transcripts.

In [3]:
from datasets import load_dataset
from unsloth import standardize_sharegpt

dataset = load_dataset("theneuralmaze/rick-and-morty-transcripts-sharegpt", split="train")
dataset = standardize_sharegpt(dataset)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
print("Number of rows: ", len(dataset))

In [ ]:
dataset[0]

Now, let's load both the model (Llama 3.1 8B) and the tokenizer.

In [ ]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=False,
    dtype=None,
)

Instead of a full finetuning, we are going to use LoRa finetuning.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    lora_alpha=64,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

The next line of code will generate a new column (`text`), that contains the data in the format needed for the finetune.

In [ ]:
from unsloth import apply_chat_template

chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""

dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
)

In [ ]:
dataset[0]

Finally, let's train for 5 epochs.

In [ ]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=5,
        output_dir="output",
        seed=0,
        report_to = "none",
    ),
)

trainer.train()

Let's test that everything works as expected before pushing the model to HF.

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

SYSTEM_PROMPT = """You are an interdimensional genius scientist named Rick Sanchez.
Be brutally honest, use sharp wit, and sprinkle in some scientific jargon.
Don't shy away from dark humor or existential truths, but always provide a solution (even if it's unconventional)."""


messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Are you a bad person?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

Push the GGUF model to HF for later download.

In [ ]:
model.push_to_hub_gguf("theneuralmaze/RickLLama-3.1-8B", tokenizer)